<a href="https://colab.research.google.com/github/dgullate/Curso-IE/blob/master/IMDB_word_embeddings_gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)

First, we read in the data with pandas, as we did in Part 1. Unlike Part 1, we now use unlabeledTrain.tsv, which contains 50,000 additional reviews with no labels. When we built the Bag of Words model in Part 1, extra unlabeled training reviews were not useful. However, since Word2Vec can learn from unlabeled data, these extra 50,000 reviews can now be used.

In [0]:
train_data_path='https://github.com/dgullate/Curso-IE/blob/master/Week_4/IMDB/data/labeledTrainData.tsv?raw=true'
test_data_path='https://github.com/dgullate/Curso-IE/blob/master/Week_4/IMDB/data/testData.tsv?raw=true'
unlabeled_data_path='https://github.com/dgullate/Curso-IE/blob/master/Week_4/IMDB/data/unlabeledTrainData.tsv?raw=true'
train = pd.read_csv(train_data_path, header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv(unlabeled_data_path, header=0, delimiter="\t", quoting=3)
raw_test_data = pd.read_csv(test_data_path, header=0, delimiter="\t", quoting=3)

The functions we write to clean the data are also similar to Part 1, although now there are a couple of differences. First, to train Word2Vec it is better not to remove stop words because the algorithm relies on the broader context of the sentence in order to produce high-quality word vectors. For this reason, we will make stop word removal optional in the functions below. It also might be better not to remove numbers, but we leave that as an exercise for the reader.

In [0]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

Next, we want a specific input format. Word2Vec expects single sentences, each one as a list of words. In other words, the input format is a list of lists.

It is not at all straightforward how to split a paragraph into sentences. There are all kinds of gotchas in natural language. English sentences can end with "?", "!", """, or ".", among other things, and spacing and capitalization are not reliable guides either. For this reason, we'll use NLTK's punkt tokenizer for sentence splitting. In order to use this, you will need to install NLTK and use nltk.download() to download the relevant training file for punkt.

In [5]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download('punkt')   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist(raw_sentence,remove_stopwords))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Let us look at one review and the output of this last function on it:

In [18]:
sentences= review_to_sentences(train['review'][7],tokenizer)
for sentence in sentences:
  print(sentence)

['i', 'watched', 'this', 'video', 'at', 'a', 'friend', 's', 'house']
['i', 'm', 'glad', 'i', 'did', 'not', 'waste', 'money', 'buying', 'this', 'one']
['the', 'video', 'cover', 'has', 'a', 'scene', 'from', 'the', 'movie', 'capricorn', 'one']
['the', 'movie', 'starts', 'out', 'with', 'several', 'clips', 'of', 'rocket', 'blow', 'ups', 'most', 'not', 'related', 'to', 'manned', 'flight']
['sibrel', 's', 'smoking', 'gun', 'is', 'a', 'short', 'video', 'clip', 'of', 'the', 'astronauts', 'preparing', 'a', 'video', 'broadcast']
['he', 'edits', 'in', 'his', 'own', 'voice', 'over', 'instead', 'of', 'letting', 'us', 'listen', 'to', 'what', 'the', 'crew', 'had', 'to', 'say']
['the', 'video', 'curiously', 'ends', 'with', 'a', 'showing', 'of', 'the', 'zapruder', 'film']
['his', 'claims', 'about', 'radiation', 'shielding', 'star', 'photography', 'and', 'others', 'lead', 'me', 'to', 'believe', 'is', 'he', 'extremely', 'ignorant', 'or', 'has', 'some', 'sort', 'of', 'ax', 'to', 'grind', 'against', 'nasa',

Now we can apply this function to prepare our data for input to Word2Vec (this will take a couple minutes):

In [25]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
n=0
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)
    n+=1
    if n%5000==0:
      print('Already parsed {} files'.format(n))
n=0
print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)
    n+=1
    if n%5000==0:
      print('Already parsed {} files'.format(n))


Parsing sentences from training set


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


Already parsed 1000 files


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Already parsed 2000 files
Already parsed 3000 files
Already parsed 4000 files
Already parsed 5000 files
Already parsed 6000 files
Already parsed 7000 files
Already parsed 8000 files
Already parsed 9000 files
Already parsed 10000 files
Already parsed 11000 files
Already parsed 12000 files
Already parsed 13000 files
Already parsed 14000 files
Already parsed 15000 files
Already parsed 16000 files
Already parsed 17000 files
Already parsed 18000 files
Already parsed 19000 files
Already parsed 20000 files
Already parsed 21000 files
Already parsed 22000 files
Already parsed 23000 files
Already parsed 24000 files
Already parsed 25000 files
Parsing sentences from unlabeled set


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behin

You may get a few warnings from BeautifulSoup about URLs in the sentences. These are nothing to worry about (although you may want to consider removing URLs when cleaning the text). 

We can take a look at the output to see how this differs from Part 1:

In [27]:
print(len(sentences))

795538


With the list of nicely parsed sentences, we're ready to train the model. There are a number of parameter choices that affect the run time and the quality of the final model that is produced. For details on the algorithms below, see the word2vec API documentation as well as the Google documentation. 

1. **Architecture**: Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results.

2. **Training algorithm**: Hierarchical softmax (default) or negative sampling. For us, the default worked well.
Downsampling of frequent words: The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.

3. **Word vector dimensionality**: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.

4. **Context / window size**: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).

5. **Worker threads**: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.

6. **Minimum word count**: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.

Choosing parameters is not easy, but once we have chosen our parameters, creating a Word2Vec model is straightforward:

In [29]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2019-05-19 16:43:09,824 : INFO : 'pattern' package not found; tag filters are not available for English
2019-05-19 16:43:09,833 : INFO : collecting all words and their counts
2019-05-19 16:43:09,834 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-05-19 16:43:09,898 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2019-05-19 16:43:09,961 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2019-05-19 16:43:10,027 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types


Training model...


2019-05-19 16:43:10,097 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2019-05-19 16:43:10,162 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2019-05-19 16:43:10,229 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2019-05-19 16:43:10,295 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2019-05-19 16:43:10,361 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2019-05-19 16:43:10,428 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2019-05-19 16:43:10,494 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2019-05-19 16:43:10,560 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2019-05-19 16:43:10,625 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keepin

## Exploring the Model Results
Let's take a look at the model we created out of our 75,000 training reviews.

The `doesnt_match` function will try to deduce which word in a set is most dissimilar from the others:

In [33]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

Our model is capable of distinguishing differences in meaning! It knows that men, women and children are more similar to each other than they are to kitchens. More exploration shows that the model is sensitive to more subtle differences in meaning, such as differences between countries and cities:

In [36]:
model.doesnt_match("france england spain madrid".split())

'madrid'

We can also use the "most_similar" function to get insight into the model's word clusters:



In [37]:
model.most_similar("man")

[('woman', 0.6234472990036011),
 ('lady', 0.6112213134765625),
 ('lad', 0.5950173139572144),
 ('monk', 0.55019611120224),
 ('chap', 0.536170482635498),
 ('guy', 0.5299067497253418),
 ('men', 0.5216111540794373),
 ('soldier', 0.5134493112564087),
 ('boy', 0.5083514451980591),
 ('millionaire', 0.5068236589431763)]

In [39]:
model.most_similar("philosophy")

[('religion', 0.6531770825386047),
 ('psychology', 0.645232081413269),
 ('philosophies', 0.6272254586219788),
 ('philosophical', 0.6229748725891113),
 ('ideals', 0.6204097270965576),
 ('alienation', 0.61898273229599),
 ('meanings', 0.6090627908706665),
 ('beliefs', 0.6027831435203552),
 ('morals', 0.6003556847572327),
 ('sexuality', 0.5974820256233215)]

In [40]:
model.most_similar("queen")

[('princess', 0.6830263733863831),
 ('stepmother', 0.6475095152854919),
 ('bride', 0.644113302230835),
 ('belle', 0.6054008603096008),
 ('eva', 0.6007740497589111),
 ('antoinette', 0.5970436334609985),
 ('victoria', 0.5903248190879822),
 ('goddess', 0.5897488594055176),
 ('mistress', 0.5894443988800049),
 ('dame', 0.5798168182373047)]

Or, more relevant for sentiment analysis:

In [41]:
model.most_similar("awful")

[('terrible', 0.7632062435150146),
 ('horrible', 0.7374094724655151),
 ('abysmal', 0.7110364437103271),
 ('dreadful', 0.7106900215148926),
 ('atrocious', 0.7014745473861694),
 ('horrendous', 0.6905228495597839),
 ('appalling', 0.6663126945495605),
 ('horrid', 0.661719799041748),
 ('lousy', 0.614993691444397),
 ('amateurish', 0.613572359085083)]